# Be able to investigate client requirements for data analysis
# 2.4 Quantitative data analysis

* mean
* median
* standard deviation
* range

## Set up

In [1]:
try:
    import micropip
    await micropip.install(["pyoliteutils", "textblob"])
except:
    pass

In [2]:
from pyoliteutils import *
import pandas as pd

## Other useful Opponent / Match Data?

### Location of opponents, and distance to Sheffield

To be able to put data on a map we will need Latitude and Lngitude,

#### Possible Data Sources : 
- https://api.postcodes.io/places?q=[query]
- Nominatum / GeoPy
    - https://geopy.readthedocs.io/en/stable/
    - https://nominatim.org/release-docs/latest/api/Search/ 
    - https://medium.com/@gopesh3652/geocoding-with-python-using-nominatim-a-beginners-guide-220b250ca48d 


## Stadium Data

In [3]:
stadiums = pd.read_csv("../data/eagles/stadiums.csv")
stadiums

,Stadium,Postcode
0,OWLERTON,S6 2DE
1,MILLMOOR,S60 1HB
2,HEADINGLEY,LS6 3BR
3,BRAMALL LANE,S2 4QX
4,HILLSBOROUGH,S6 1SW
5,OAKWELL,S71 1ET
6,BELLE VUE,DN4 5DX
7,SALTERGATE,S40 4SX
8,TATTERSFIELD,DN4 5JW
9,THRUM HALL,HX1 4LG


## Adding the Latitude & Longitude

In [4]:
import postcodes_io_api
class Api2(postcodes_io_api.Api):
        def places_query(self, place):
            """
            This method returns data for post_code
            * **:param postcode** - postcode to check i.e. 'SW112EF'
            * **::return** - postcode detailed data
            ```
              data = api.places_query('Sheffield')
        
            ```
            """
            url = '/places?q={place}'.format(place=place)
            response = self._make_request('GET', url)
            data = self._parse_json_data(response.content.decode('utf-8'))
            return data

api  = Api2()

def get_place(place):
    latitude = None
    longitude = None
    
    data = api.places_query(place)
    #print("data", data)
    
    if data["status"] == 200 and len(data["result"]):        
        if "latitude" in data["result"][0]:
            latitude = data["result"][0]["latitude"]
            longitude = data["result"][0]["longitude"]
    #print("latitude, longitude", latitude, longitude)
    return latitude, longitude

def get_latlong(postcode):
    latitude = None
    longitude = None
    
    data = api.get_postcode(postcode)
    #print("data", data)
    if data["status"] != 200 :
        data = api.get_outcode(postcode)
    
    if data["status"] == 200 :        
        if "latitude" in data["result"]:
            latitude = data["result"]["latitude"]
            longitude = data["result"]["longitude"]
    #print("latitude, longitude", latitude, longitude)
    return latitude, longitude
    
def get_latlongs(df):
    if ("Latitude" not in df) and ("Postcode" in df):    
      try:
        df[["Latitude", "Longitude"]] = df.apply(
            lambda row: get_latlong(row["Postcode"]), axis=1, result_type="expand"
        )
      except Exception as e:
          print('Postcode Conversion failed : '+ str(e))
    return df

def get_places(df, field_name):
    if ("Latitude" not in df) and (field_name in df):    
      try:
        df[[field_name + " Latitude", field_name + " Longitude"]] = df.apply(
            lambda row: get_place(row[field_name]), axis=1, result_type="expand"
        )
      except Exception as e:
          print('Place  Conversion failed : '+ str(e))
    return df

In [5]:
stadiums = get_latlongs(stadiums)
stadiums

,Stadium,Postcode,Latitude,Longitude
0,OWLERTON,S6 2DE,53.406031,-1.493303
1,MILLMOOR,S60 1HB,53.428758,-1.369397
2,HEADINGLEY,LS6 3BR,53.816081,-1.580617
3,BRAMALL LANE,S2 4QX,53.371341,-1.469862
4,HILLSBOROUGH,S6 1SW,53.410844,-1.500859
5,OAKWELL,S71 1ET,53.552266,-1.468631
6,BELLE VUE,DN4 5DX,53.517626,-1.108750
7,SALTERGATE,S40 4SX,53.238963,-1.434745
8,TATTERSFIELD,DN4 5JW,53.509088,-1.113845
9,THRUM HALL,HX1 4LG,53.721864,-1.884001


In [6]:
#https://towardsdatascience.com/calculating-distance-between-two-geolocations-in-python-26ad3afe287b

sheffield_latlong = get_place("Sheffield")
import haversine as hs

#stadiums['dist_from_sheffield'] = hs.haversine(sheffield_latlong, (stadiums["Latitude"], stadiums["Longitude"]), unit=Unit.MILES)

stadiums['Miles from Sheffield'] = stadiums.apply(
    lambda row: hs.haversine(sheffield_latlong, (row["Latitude"], row["Longitude"]), unit=hs.Unit.MILES), axis=1, result_type="expand"
)
stadiums

,Stadium,Postcode,Latitude,Longitude,Miles from Sheffield
0,OWLERTON,S6 2DE,53.406031,-1.493303,1.918934
1,MILLMOOR,S60 1HB,53.428758,-1.369397,5.185172
2,HEADINGLEY,LS6 3BR,53.816081,-1.580617,30.307328
3,BRAMALL LANE,S2 4QX,53.371341,-1.469862,0.773637
4,HILLSBOROUGH,S6 1SW,53.410844,-1.500859,2.367659
5,OAKWELL,S71 1ET,53.552266,-1.468631,11.729022
6,BELLE VUE,DN4 5DX,53.517626,-1.108750,17.501780
7,SALTERGATE,S40 4SX,53.238963,-1.434745,10.015773
8,TATTERSFIELD,DN4 5JW,53.509088,-1.113845,17.015450
9,THRUM HALL,HX1 4LG,53.721864,-1.884001,28.993035


In [7]:
#Save for later 
stadiums.to_csv("../data/eagles/stadiums_with_latlong.csv", index=False)

## Crowd / Attendance Data

In [8]:
crowds = pd.read_csv("../data/eagles/crowds.csv")
## https://datascienceparichay.com/article/pandas-extract-year-from-datetime-column/
crowds

,Date,Opponents,Attendance
0,02/09/1984,Rochdale,1425
1,09/09/1984,Fulham,1145
2,23/09/1984,Salford,1159
3,14/10/1984,Runcorn,1076
4,28/10/1984,Bridgend,826
...,...,...,...
583,24/07/2022,Workington,644
584,07/08/2022,Whitehaven,727
585,12/08/2022,Halifax,569
586,02/09/2022,Featherstone,912


## Add the distance to sheffield

In [9]:
def team_location(opponent):
    location = opponent.replace("St Pats", "")
    location = location.replace("R-", "")
    location = location.replace("KR", "")
    location = location.replace("East", "")
    location = location.replace("Met", "")
    location = location.replace("Crusaders", "Wrexham") # or Bridgeend
    location = location.replace("Skolars", "London ")
    location = location.strip(" ")
    return location
    
crowds['Town'] = crowds.apply(
    lambda row: team_location(row["Opponents"]), axis=1, result_type="expand"
)
crowds

,Date,Opponents,Attendance,Town
0,02/09/1984,Rochdale,1425,Rochdale
1,09/09/1984,Fulham,1145,Fulham
2,23/09/1984,Salford,1159,Salford
3,14/10/1984,Runcorn,1076,Runcorn
4,28/10/1984,Bridgend,826,Bridgend
...,...,...,...,...
583,24/07/2022,Workington,644,Workington
584,07/08/2022,Whitehaven,727,Whitehaven
585,12/08/2022,Halifax,569,Halifax
586,02/09/2022,Featherstone,912,Featherstone


In [10]:
crowds = get_places(crowds, "Town")
crowds

,Date,Opponents,Attendance,Town,Town Latitude,Town Longitude
0,02/09/1984,Rochdale,1425,Rochdale,53.616965,-2.156708
1,09/09/1984,Fulham,1145,Fulham,51.471626,-0.211248
2,23/09/1984,Salford,1159,Salford,53.710701,-2.098607
3,14/10/1984,Runcorn,1076,Runcorn,53.341713,-2.731298
4,28/10/1984,Bridgend,826,Bridgend,56.295700,-2.978223
...,...,...,...,...,...,...
583,24/07/2022,Workington,644,Workington,54.643374,-3.549369
584,07/08/2022,Whitehaven,727,Whitehaven,54.550728,-3.587463
585,12/08/2022,Halifax,569,Halifax,53.722830,-1.860176
586,02/09/2022,Featherstone,912,Featherstone,53.674631,-1.359178


In [11]:
# filter for na/s and location for teams
crowds[crowds['Town Latitude'].isnull()]



,Date,Opponents,Attendance,Town,Town Latitude,Town Longitude
143,14/10/1992,Australia,5500,Australia,NaN,NaN
176,26/10/1994,Australia,7450,Australia,NaN,NaN
205,15/06/1996,Paris,5350,Paris,NaN,NaN
211,16/03/1997,Paris,4213,Paris,NaN,NaN
377,07/03/2008,Lezignan,379,Lezignan,NaN,NaN
389,08/03/2009,Toulouse,1554,Toulouse,NaN,NaN
394,23/05/2009,Toulouse,809,Toulouse,NaN,NaN
411,15/05/2010,Toulouse,979,Toulouse,NaN,NaN
420,26/03/2011,Toulouse,1076,Toulouse,NaN,NaN
514,12/02/2017,Toulouse,442,Toulouse,NaN,NaN


In [15]:
#crowds = crowds[crowds['Town Latitude'].isnull() == False]
crowds = crowds.dropna()
crowds

,Date,Opponents,Attendance,Town,Town Latitude,Town Longitude,Opponents Miles from Sheffield
0,02/09/1984,Rochdale,1425,Rochdale,53.616965,-2.156708,32.593325
1,09/09/1984,Fulham,1145,Fulham,51.471626,-0.211248,142.253540
2,23/09/1984,Salford,1159,Salford,53.710701,-2.098607,34.398366
3,14/10/1984,Runcorn,1076,Runcorn,53.341713,-2.731298,52.141615
4,28/10/1984,Bridgend,826,Bridgend,56.295700,-2.978223,210.040982
...,...,...,...,...,...,...,...
583,24/07/2022,Workington,644,Workington,54.643374,-3.549369,121.342450
584,07/08/2022,Whitehaven,727,Whitehaven,54.550728,-3.587463,118.025449
585,12/08/2022,Halifax,569,Halifax,53.722830,-1.860176,28.483803
586,02/09/2022,Featherstone,912,Featherstone,53.674631,-1.359178,20.677304


In [16]:
crowds['Opponents Miles from Sheffield'] = crowds.apply(
    lambda row: hs.haversine(sheffield_latlong, (row["Town Latitude"], row["Town Longitude"]), unit=hs.Unit.MILES), axis=1, result_type="expand"
)
crowds


,Date,Opponents,Attendance,Town,Town Latitude,Town Longitude,Opponents Miles from Sheffield
0,02/09/1984,Rochdale,1425,Rochdale,53.616965,-2.156708,32.593325
1,09/09/1984,Fulham,1145,Fulham,51.471626,-0.211248,142.253540
2,23/09/1984,Salford,1159,Salford,53.710701,-2.098607,34.398366
3,14/10/1984,Runcorn,1076,Runcorn,53.341713,-2.731298,52.141615
4,28/10/1984,Bridgend,826,Bridgend,56.295700,-2.978223,210.040982
...,...,...,...,...,...,...,...
583,24/07/2022,Workington,644,Workington,54.643374,-3.549369,121.342450
584,07/08/2022,Whitehaven,727,Whitehaven,54.550728,-3.587463,118.025449
585,12/08/2022,Halifax,569,Halifax,53.722830,-1.860176,28.483803
586,02/09/2022,Featherstone,912,Featherstone,53.674631,-1.359178,20.677304


## Get Usable Date Information

In [17]:
crowds['Date'] = pd.to_datetime(crowds['Date'], format="%d/%m/%Y")
crowds['Year'] = crowds['Date'].dt.year
crowds['Day'] = crowds['Date'].dt.day_name()
crowds['Month'] = crowds['Date'].dt.month
crowds['Month Name'] = crowds['Date'].dt.month_name()
crowds

,Date,Opponents,Attendance,Town,Town Latitude,Town Longitude,Opponents Miles from Sheffield,Year,Day,Month,Month Name
0,1984-09-02,Rochdale,1425,Rochdale,53.616965,-2.156708,32.593325,1984,Sunday,9,September
1,1984-09-09,Fulham,1145,Fulham,51.471626,-0.211248,142.253540,1984,Sunday,9,September
2,1984-09-23,Salford,1159,Salford,53.710701,-2.098607,34.398366,1984,Sunday,9,September
3,1984-10-14,Runcorn,1076,Runcorn,53.341713,-2.731298,52.141615,1984,Sunday,10,October
4,1984-10-28,Bridgend,826,Bridgend,56.295700,-2.978223,210.040982,1984,Sunday,10,October
...,...,...,...,...,...,...,...,...,...,...,...
583,2022-07-24,Workington,644,Workington,54.643374,-3.549369,121.342450,2022,Sunday,7,July
584,2022-08-07,Whitehaven,727,Whitehaven,54.550728,-3.587463,118.025449,2022,Sunday,8,August
585,2022-08-12,Halifax,569,Halifax,53.722830,-1.860176,28.483803,2022,Friday,8,August
586,2022-09-02,Featherstone,912,Featherstone,53.674631,-1.359178,20.677304,2022,Friday,9,September


## Saving for later

In [18]:
crowds.to_csv("../data/eagles/crowds_clean.csv")

### Weather in Sheffield

Could do daily weather at the correct stadium location but monthly for Sheffield's Weather station will be easier at first

#### Possible Data Sources : 

- https://www.metoffice.gov.uk/research/climate/maps-and-data/historic-station-data saved as [text](../data/eagles/sheffield_montly_weather.csv)
- https://api.openweathermap.org/data/3.0/onecall/timemachine?lat=39.099724&lon=-94.578331&dt=1643803200&appid=b112bddb3ca3876644b7c695768ae96d

In [19]:
sheffield_monthly_weather = pd.read_csv("../data/eagles/sheffield_monthly_weather.csv")
sheffield_monthly_weather

,yyyy,mm,tmax degC,tmin degC,af days,rain mm,sun hours
0,1883,1,6.3,1.7,6,122.1,---
1,1883,2,8,2.8,2,69.8,---
2,1883,3,4.8,-1.6,23,29.6,---
3,1883,4,12.2,3.8,2,74,---
4,1883,5,14.7,6.2,0,31.2,---
...,...,...,...,...,...,...,...
1689,2023,10,14.9,8.9,0,203.2,77.5# Provisional
1690,2023,11,9.4,5,2,108.2,68.1# Provisional
1691,2023,12,9.1,* 4.7,5,174.3,* 31.5# Provisional
1692,2024,1,7,2.6,6,72.4,54.9# Provisional


In [20]:
sheffield_monthly_weather.rename(columns={
    'yyyy' : "Year",
    'mm' : "Month",
    'tmax degC' : "Max Temperature C",
    'tmin degC' : "Min Temperature C",
    'rain mm' : "Rain mm",
},inplace=True)
sheffield_monthly_weather

,Year,Month,Max Temperature C,Min Temperature C,af days,Rain mm,sun hours
0,1883,1,6.3,1.7,6,122.1,---
1,1883,2,8,2.8,2,69.8,---
2,1883,3,4.8,-1.6,23,29.6,---
3,1883,4,12.2,3.8,2,74,---
4,1883,5,14.7,6.2,0,31.2,---
...,...,...,...,...,...,...,...
1689,2023,10,14.9,8.9,0,203.2,77.5# Provisional
1690,2023,11,9.4,5,2,108.2,68.1# Provisional
1691,2023,12,9.1,* 4.7,5,174.3,* 31.5# Provisional
1692,2024,1,7,2.6,6,72.4,54.9# Provisional


In [21]:
#result = pd.concat([crowds, sheffield_monthly_weather], axis=1, join="inner")
crowds_with_monthly_weather = pd.merge(crowds, sheffield_monthly_weather, how="left", on=["Year", "Month"])
crowds_with_monthly_weather.to_csv("../data/eagles/crowds_with_monthly_weather.csv", index=False)
crowds_with_monthly_weather

,Date,Opponents,Attendance,Town,Town Latitude,Town Longitude,Opponents Miles from Sheffield,Year,Day,Month,Month Name,Max Temperature C,Min Temperature C,af days,Rain mm,sun hours
0,1984-09-02,Rochdale,1425,Rochdale,53.616965,-2.156708,32.593325,1984,Sunday,9,September,16.7,10.4,0,117.6,86.1
1,1984-09-09,Fulham,1145,Fulham,51.471626,-0.211248,142.253540,1984,Sunday,9,September,16.7,10.4,0,117.6,86.1
2,1984-09-23,Salford,1159,Salford,53.710701,-2.098607,34.398366,1984,Sunday,9,September,16.7,10.4,0,117.6,86.1
3,1984-10-14,Runcorn,1076,Runcorn,53.341713,-2.731298,52.141615,1984,Sunday,10,October,14.3,7.7,0,82.7,75.6
4,1984-10-28,Bridgend,826,Bridgend,56.295700,-2.978223,210.040982,1984,Sunday,10,October,14.3,7.7,0,82.7,75.6
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
569,2022-07-24,Workington,644,Workington,54.643374,-3.549369,121.342450,2022,Sunday,7,July,23.2,14.5,0,23.8,187.8*
570,2022-08-07,Whitehaven,727,Whitehaven,54.550728,-3.587463,118.025449,2022,Sunday,8,August,23.9,13.6,0,56.4,234.5*
571,2022-08-12,Halifax,569,Halifax,53.722830,-1.860176,28.483803,2022,Friday,8,August,23.9,13.6,0,56.4,234.5*
572,2022-09-02,Featherstone,912,Featherstone,53.674631,-1.359178,20.677304,2022,Friday,9,September,18.4,11.1,0,50,124.8*


In [22]:
 crowds_with_monthly_weather[crowds_with_monthly_weather.isna().any(axis=1)]

,Date,Opponents,Attendance,Town,Town Latitude,Town Longitude,Opponents Miles from Sheffield,Year,Day,Month,Month Name,Max Temperature C,Min Temperature C,af days,Rain mm,sun hours


## Survey Data

In [23]:
questionnaire = pd.read_csv("../data/eagles/questionnaire.csv")
questionnaire

,Timestamp,Are you:,When did you first attend a Sheffield Eagles game?,Would you class yourself as:,When did you last attend a game (home or away)?,What made you first start watching the Sheffield Eagles?,What is your favourite memory of the Sheffield Eagles?,"Do you have any Sheffield Eagles memorabilia - old photographs, match day programmes, stickers, shirts, press cuttings etc - that you'd be willing to share with the project? If so please provide brief details.","Can you quickly describe what the Sheffield Eagles means to you, this could be because you are a supporter, a player, a sponsor or an organisation that works with the Club.",What are some of the stories that are part of the history of the Sheffield Eagles that you think deserve sharing to a wider audience?,...,"What do you see as the possible benefits of a project like this, for you personally, the organisation you work for, rugby league in general or the club specifically?",Do you think there is a need for a project like this? If so could you say what that need is.,Please let us have any other comments about the project.,Are You?,How old are you?,How would you describe yourself,Could you provide your postcode? This will help us understand where people are responding from.,Unnamed: 18,Unnamed: 19,Unnamed: 20
0,9/1/2021 14:05:26,A player or member of club staff,2011- 2021,Someone who attends games home or away on a re...,29 August 2021,"Student in Sheffield, was a St. Helens fan and...",Winning the 1895 Cup,No,"My life really! Spend most of my time, both wo...","1895 Cup Win, The Supporters helping save the ...",...,"Reaching a wider audience, gaining better 'cut...",Absolutely a club that has a short but rich hi...,NaN,Male,25-39,White,S70,NaN,NaN,NaN
1,9/2/2021 9:23:44,A current Sheffield Eagles fan who attends games,1984 - 1990,Someone who has a season ticket and attends al...,Last Sunday away at London,I had already been watching the game for a few...,Challenge Cup win in 98,Yes lots of shirts.,The club has been a major part of my life for ...,It would be good for some of the newer fans to...,...,It will educate people thinking of supporting ...,To give the club a higher profile in the city ...,"No, I think everything has been covered.",Male,50-59,White,S2,NaN,NaN,NaN
2,9/2/2021 9:25:42,A current Sheffield Eagles fan who attends games,1991 - 2000,Someone who has a season ticket and attends al...,Win over Oldham at home,Something the whole family could do together,Winning league for the second time,Lots of photos,Togetherness & passion,Endurance with little support from Council and...,...,Ability to show what the Eagles are about and ...,Yes to preserve the history for future and to ...,NaN,Male,50-59,White,S2,NaN,NaN,NaN
3,9/2/2021 10:05:39,A current Sheffield Eagles fan who attends games,1984 - 1990,Someone who attends games home or away on a re...,Batley Away match,1988,Challenge cup final 1998,"Got replica shirts , Programmes , Signed shirt",local rugby league Club,NaN,...,NaN,"Yes , Every club should have a heritage site",NaN,Male,50-59,White,S70,NaN,NaN,NaN
4,9/2/2021 10:53:30,A current Sheffield Eagles fan who attends games,1991 - 2000,Someone who has a season ticket and attends al...,Two weeks ago - Batley away.,Mum and Dad were big fans.,Back-to-back Grand Final wins.,Nothing from the past.,The Eagles mean a lot to me as I've met some a...,The years between 2014-2017 where we had a dif...,...,It uncovers new stories that make Rugby League...,Yes I think there is a need as there will be p...,NaN,Male,16-25,White,S5,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
122,9/27/2021 8:00:25,A player or member of club staff,NaN,A current or former player,NaN,NaN,Winning the 2nd division final at Old Trafford,Yes,A brilliant closenit club,NaN,...,NaN,NaN,NaN,Male,60-69,White,WF10,NaN,NaN,NaN
123,9/27/2021 10:13:35,A Sheffield Eagles fan who no longer attends g...,1991 - 2000,A current or former player,2014,"Fat

In [24]:

questionnaire.rename(columns={
    'Could you provide your postcode? This will help us understand where people are responding from. ':"Postcode",
},inplace=True)

questionnaire


,Timestamp,Are you:,When did you first attend a Sheffield Eagles game?,Would you class yourself as:,When did you last attend a game (home or away)?,What made you first start watching the Sheffield Eagles?,What is your favourite memory of the Sheffield Eagles?,"Do you have any Sheffield Eagles memorabilia - old photographs, match day programmes, stickers, shirts, press cuttings etc - that you'd be willing to share with the project? If so please provide brief details.","Can you quickly describe what the Sheffield Eagles means to you, this could be because you are a supporter, a player, a sponsor or an organisation that works with the Club.",What are some of the stories that are part of the history of the Sheffield Eagles that you think deserve sharing to a wider audience?,...,"What do you see as the possible benefits of a project like this, for you personally, the organisation you work for, rugby league in general or the club specifically?",Do you think there is a need for a project like this? If so could you say what that need is.,Please let us have any other comments about the project.,Are You?,How old are you?,How would you describe yourself,Postcode,Unnamed: 18,Unnamed: 19,Unnamed: 20
0,9/1/2021 14:05:26,A player or member of club staff,2011- 2021,Someone who attends games home or away on a re...,29 August 2021,"Student in Sheffield, was a St. Helens fan and...",Winning the 1895 Cup,No,"My life really! Spend most of my time, both wo...","1895 Cup Win, The Supporters helping save the ...",...,"Reaching a wider audience, gaining better 'cut...",Absolutely a club that has a short but rich hi...,NaN,Male,25-39,White,S70,NaN,NaN,NaN
1,9/2/2021 9:23:44,A current Sheffield Eagles fan who attends games,1984 - 1990,Someone who has a season ticket and attends al...,Last Sunday away at London,I had already been watching the game for a few...,Challenge Cup win in 98,Yes lots of shirts.,The club has been a major part of my life for ...,It would be good for some of the newer fans to...,...,It will educate people thinking of supporting ...,To give the club a higher profile in the city ...,"No, I think everything has been covered.",Male,50-59,White,S2,NaN,NaN,NaN
2,9/2/2021 9:25:42,A current Sheffield Eagles fan who attends games,1991 - 2000,Someone who has a season ticket and attends al...,Win over Oldham at home,Something the whole family could do together,Winning league for the second time,Lots of photos,Togetherness & passion,Endurance with little support from Council and...,...,Ability to show what the Eagles are about and ...,Yes to preserve the history for future and to ...,NaN,Male,50-59,White,S2,NaN,NaN,NaN
3,9/2/2021 10:05:39,A current Sheffield Eagles fan who attends games,1984 - 1990,Someone who attends games home or away on a re...,Batley Away match,1988,Challenge cup final 1998,"Got replica shirts , Programmes , Signed shirt",local rugby league Club,NaN,...,NaN,"Yes , Every club should have a heritage site",NaN,Male,50-59,White,S70,NaN,NaN,NaN
4,9/2/2021 10:53:30,A current Sheffield Eagles fan who attends games,1991 - 2000,Someone who has a season ticket and attends al...,Two weeks ago - Batley away.,Mum and Dad were big fans.,Back-to-back Grand Final wins.,Nothing from the past.,The Eagles mean a lot to me as I've met some a...,The years between 2014-2017 where we had a dif...,...,It uncovers new stories that make Rugby League...,Yes I think there is a need as there will be p...,NaN,Male,16-25,White,S5,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
122,9/27/2021 8:00:25,A player or member of club staff,NaN,A current or former player,NaN,NaN,Winning the 2nd division final at Old Trafford,Yes,A brilliant closenit club,NaN,...,NaN,NaN,NaN,Male,60-69,White,WF10,NaN,NaN,NaN
123,9/27/2021 10:13:35,A Sheffield Eagles fan who no longer attends g...,1991 - 2000,A current or former player,2014,"Father managing DVS, would go to work with him.",2013 grand final win.,NaN,A route into ru

In [25]:


questionnaire_quantitive = questionnaire[[
    questionnaire.columns[1],
    questionnaire.columns[2],
    questionnaire.columns[3],
    questionnaire.columns[14],
    questionnaire.columns[15],
    questionnaire.columns[16],
    questionnaire.columns[17]
]]
questionnaire_quantitive

,Are you:,When did you first attend a Sheffield Eagles game?,Would you class yourself as:,Are You?,How old are you?,How would you describe yourself,Postcode
0,A player or member of club staff,2011- 2021,Someone who attends games home or away on a re...,Male,25-39,White,S70
1,A current Sheffield Eagles fan who attends games,1984 - 1990,Someone who has a season ticket and attends al...,Male,50-59,White,S2
2,A current Sheffield Eagles fan who attends games,1991 - 2000,Someone who has a season ticket and attends al...,Male,50-59,White,S2
3,A current Sheffield Eagles fan who attends games,1984 - 1990,Someone who attends games home or away on a re...,Male,50-59,White,S70
4,A current Sheffield Eagles fan who attends games,1991 - 2000,Someone who has a season ticket and attends al...,Male,16-25,White,S5
...,...,...,...,...,...,...,...
122,A player or member of club staff,NaN,A current or former player,Male,60-69,White,WF10
123,A Sheffield Eagles fan who no longer attends g...,1991 - 2000,A current or former player,Male,25-39,White,HU6
124,A current Sheffield Eagles fan who attends games,1991 - 2000,Someone who has a season ticket and attends al...,Male,60-69,White,S11
125,A former player,2001 - 2010,A current or former player,Male,25-39,White,S74


In [26]:
questionnaire_quantitive = get_latlongs(questionnaire_quantitive)
questionnaire_quantitive

/tmp/ipykernel_3255/3190890784.py:53: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df[["Latitude", "Longitude"]] = df.apply(
/tmp/ipykernel_3255/3190890784.py:53: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df[["Latitude", "Longitude"]] = df.apply(


,Are you:,When did you first attend a Sheffield Eagles game?,Would you class yourself as:,Are You?,How old are you?,How would you describe yourself,Postcode,Latitude,Longitude
0,A player or member of club staff,2011- 2021,Someone who attends games home or away on a re...,Male,25-39,White,S70,53.541201,-1.474917
1,A current Sheffield Eagles fan who attends games,1984 - 1990,Someone who has a season ticket and attends al...,Male,50-59,White,S2,53.369602,-1.447879
2,A current Sheffield Eagles fan who attends games,1991 - 2000,Someone who has a season ticket and attends al...,Male,50-59,White,S2,53.369602,-1.447879
3,A current Sheffield Eagles fan who attends games,1984 - 1990,Someone who attends games home or away on a re...,Male,50-59,White,S70,53.541201,-1.474917
4,A current Sheffield Eagles fan who attends games,1991 - 2000,Someone who has a season ticket and attends al...,Male,16-25,White,S5,53.422264,-1.463306
...,...,...,...,...,...,...,...,...,...
122,A player or member of club staff,NaN,A current or former player,Male,60-69,White,WF10,53.723921,-1.347748
123,A Sheffield Eagles fan who no longer attends g...,1991 - 2000,A current or former player,Male,25-39,White,HU6,53.781038,-0.371421
124,A current Sheffield Eagles fan who attends games,1991 - 2000,Someone who has a season ticket and attends al...,Male,60-69,White,S11,53.360769,-1.507895
125,A former player,2001 - 2010,A current or former player,Male,25-39,White,S74,53.500973,-1.441331


In [27]:
questionnaire_quantitive.to_csv("../data/eagles/questionnaire_quantitive.csv", index=False)